In [12]:
import boto3

with open('trigger.py','r') as f:
    function_code = f.read()

function_name = 'LambdaTrigger'
runtime='python3.8'
handler='lambda_function.lambda_trigger' 


import json
iam_client=boto3.client('iam',region_name='us-east-1')

In [ ]:
#policy name: CourseLambdaTriggerPolicy
#role -> CourseLambdaExecutionRoleTrigger

In [4]:
role_arn = 'arn:aws:iam::732077674619:role/CourseLambdaExecutionRoleTrigger'

In [13]:
print(function_code)

import json

def lambda_trigger(event,context):
    s3_object_key = event['Records'][0]['s3']['object']['key']
    print(f"File: {s3_object_key}")
    return s3_object_key


In [69]:
import zipfile
import io
lambda_client=boto3.client('lambda',region_name='us-east-1')

In [18]:
with io.BytesIO() as deployment_package:
    with zipfile.ZipFile(deployment_package,'w') as zipf:
        zipf.writestr('lambda_function.py',function_code)
    create_function_response = lambda_client.create_function(
    FunctionName=function_name,
    Runtime=runtime,
    Role = role_arn,
    Handler = handler,
    Code = {'ZipFile':deployment_package.getvalue()}
    )
    



In [56]:
create_function_response

{'ResponseMetadata': {'RequestId': 'ac72eb89-a13b-4322-932d-7ce4c6186db1',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Thu, 09 Nov 2023 14:29:25 GMT',
   'content-type': 'application/json',
   'content-length': '1216',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ac72eb89-a13b-4322-932d-7ce4c6186db1'},
  'RetryAttempts': 0},
 'FunctionName': 'LambdaTrigger',
 'FunctionArn': 'arn:aws:lambda:us-east-1:732077674619:function:LambdaTrigger',
 'Runtime': 'python3.8',
 'Role': 'arn:aws:iam::732077674619:role/CourseLambdaExecutionRoleTrigger',
 'Handler': 'lambda_function.lambda_trigger',
 'CodeSize': 305,
 'Description': '',
 'Timeout': 3,
 'MemorySize': 128,
 'LastModified': '2023-11-09T14:29:25.415+0000',
 'CodeSha256': 'VEbtlSwL4EpxCfPBnOOxY0Y66K4pwHGI/t59+aKvoGU=',
 'Version': '$LATEST',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': '674ead9a-0928-4a2f-961b-975cc8d04819',
 'State': 'Pending',
 'StateReason': 'The function is being created.',
 'StateReasonCode

In [58]:
s3_client=boto3.client('s3',region_name='us-east-1')

In [66]:
bucket_name = 'bhargava-new-bucket'
s3_client.create_bucket(Bucket = bucket_name)

{'ResponseMetadata': {'RequestId': 'Z2AY580BMKGN8J09',
  'HostId': 'M4i10hFALBhAFuuNEuPQ36X4ZxSx9VmlDSUO0K2ar73z9LeB0k02q+hpbUsilhInO3Hi3NLzHr8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'M4i10hFALBhAFuuNEuPQ36X4ZxSx9VmlDSUO0K2ar73z9LeB0k02q+hpbUsilhInO3Hi3NLzHr8=',
   'x-amz-request-id': 'Z2AY580BMKGN8J09',
   'date': 'Thu, 09 Nov 2023 15:17:18 GMT',
   'location': '/bhargava-new-bucket',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/bhargava-new-bucket'}

In [67]:
bucket_arn = 'arn:aws:s3:::bhargava-new-bucket'

In [71]:
#adding the permission to invoke the function whenever an object is added to s3
lambda_client.add_permission(FunctionName=function_name,
                            StatementId ='Id1',
                            Action = 'lambda:InvokeFunction',
                            Principal ='s3.amazonaws.com',
                            SourceArn = bucket_arn)

{'ResponseMetadata': {'RequestId': '108cd215-66f5-41db-b123-4f2e424f70eb',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Thu, 09 Nov 2023 15:19:04 GMT',
   'content-type': 'application/json',
   'content-length': '303',
   'connection': 'keep-alive',
   'x-amzn-requestid': '108cd215-66f5-41db-b123-4f2e424f70eb'},
  'RetryAttempts': 0},
 'Statement': '{"Sid":"Id1","Effect":"Allow","Principal":{"Service":"s3.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:732077674619:function:LambdaTrigger","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:s3:::bhargava-new-bucket"}}}'}

In [72]:
#adding the trigger

event_configuration = {
'LambdaFunctionConfigurations':[
{'LambdaFunctionArn':create_function_response['FunctionArn'],
'Events':['s3:ObjectCreated:*']}
]
}

In [73]:
s3_client.put_bucket_notification_configuration(Bucket=bucket_name,
                                                NotificationConfiguration=event_configuration
                                               )

{'ResponseMetadata': {'RequestId': 'T2D4XYJ0FCJH4564',
  'HostId': 'yGh6rhjG2/A2rIymym5FMyuiR/jaX/SR6PxjL083SYBQP01ySjtfcEM1rTrVN5I2e3yemWSwTOU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'yGh6rhjG2/A2rIymym5FMyuiR/jaX/SR6PxjL083SYBQP01ySjtfcEM1rTrVN5I2e3yemWSwTOU=',
   'x-amz-request-id': 'T2D4XYJ0FCJH4564',
   'date': 'Thu, 09 Nov 2023 15:27:21 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [75]:
with open('Text_file.txt','w') as file:
    file.write("this is a trigger test")

In [79]:
#the upload should trigger the lambda
s3_client.upload_file(Filename='Text_file.txt',Bucket=bucket_name,Key='test_file.txt')

In [80]:
#checking if it triggerd the lambda

In [83]:
cloudwatch_logs=boto3.client('logs',region_name='us-east-1')

In [85]:
response = cloudwatch_logs.describe_log_groups()

In [86]:
response

{'logGroups': [{'logGroupName': '/aws/lambda/LambdaTrigger',
   'creationTime': 1699543849604,
   'metricFilterCount': 0,
   'arn': 'arn:aws:logs:us-east-1:732077674619:log-group:/aws/lambda/LambdaTrigger:*',
   'storedBytes': 0}],
 'ResponseMetadata': {'RequestId': '8f9dc037-984a-4f74-bad3-03a54444d219',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8f9dc037-984a-4f74-bad3-03a54444d219',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '297',
   'date': 'Thu, 09 Nov 2023 15:32:57 GMT'},
  'RetryAttempts': 0}}

In [91]:
log_streams = cloudwatch_logs.describe_log_streams(logGroupName=f'/aws/lambda/{function_name}',
                                                  orderBy = 'LastEventTime',
                                                  descending=True)['logStreams']

In [92]:
log_streams

[{'logStreamName': '2023/11/09/[$LATEST]1055f00283a54bdc9081a88efa0f4bac',
  'creationTime': 1699543849636,
  'firstEventTimestamp': 1699543843161,
  'lastEventTimestamp': 1699543843279,
  'lastIngestionTime': 1699543849646,
  'uploadSequenceToken': '49039859566789708481000097538180275559085243269642784703',
  'arn': 'arn:aws:logs:us-east-1:732077674619:log-group:/aws/lambda/LambdaTrigger:log-stream:2023/11/09/[$LATEST]1055f00283a54bdc9081a88efa0f4bac',
  'storedBytes': 0}]

In [97]:
result = cloudwatch_logs.get_log_events(logGroupName=f'/aws/lambda/{function_name}',
                              logStreamName = log_streams[0]['logStreamName'])

In [108]:
result['events'][2]['message'] #-> this is the string which was returned by our function when it was called

'File: test_file.txt\n'

In [ ]:
# that's means the trigger worked